In [6]:
import pandas as pd
import re
from pathlib import Path
from unidecode import unidecode

In [7]:
# === Caminhos ===
DADOS_DIR = Path("Dados/")
RESULTADOS_DIR = Path("Resultados/")
RESULTADOS_DIR.mkdir(exist_ok=True)

In [8]:
# Corrige aspas duplicadas antes de ler
caminho = Path("Dados/NetflixViewingHistory.csv")
rows = []
with open(caminho, "r", encoding="latin1") as f:
    header = f.readline()  # deve ser Title,Date
    for line in f:
        line = line.strip()
        if not line:
            continue
        # remove aspas externas
        if line.startswith('"') and line.endswith('"'):
            line = line[1:-1]
        # colapsa aspas duplas internas
        line = line.replace('""', '"')
        # separa pelo ÚLTIMO vírgula (título pode ter vírgulas; data não)
        try:
            title, date = line.rsplit(",", 1)
        except ValueError:
            # linha estranha: pula ou loga
            continue
        title = title.strip().strip('"')
        date = date.strip().strip('"')
        rows.append((title, date))

netflix = pd.DataFrame(rows, columns=["Title", "Date"])
netflix["Date"] = pd.to_datetime(netflix["Date"], errors="coerce")
print(netflix.head())

                                             Title       Date
0  Doom at Your Service: Limited Series: Episode 2 2025-10-19
1  Doom at Your Service: Limited Series: Episode 1 2025-10-19
2             Brooklyn Nine-Nine: Season 7: Ransom 2025-10-18
3          Love Next Door: Limited Series: The End 2025-10-17
4   Love Next Door: Limited Series: Bravo, My Life 2025-10-17


C:\Users\arviz\AppData\Local\Temp\ipykernel_35220\526692256.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  netflix["Date"] = pd.to_datetime(netflix["Date"], errors="coerce")


In [9]:
# === Função de limpeza dos títulos ===
def limpar_titulos(df, coluna):
    def clean_title(title):
        if not isinstance(title, str) or not title.strip():
            return ""
        t = title.lower().strip()
        t = unidecode(t)  # remove acentos
        # remove partes de temporada/episódio
        t = re.sub(r":\s*(season|seasons|part|episode|ep|chapter|capitulo)\s*\d+", "", t)
        # remove tudo após o segundo ":"
        t = re.split(r":", t, maxsplit=2)[0].strip()
        # remove conteúdo entre parênteses
        t = re.sub(r"[\(\[].*?[\)\]]", "", t)
        # remove caracteres especiais
        t = re.sub(r"[^a-z0-9\s]", "", t)
        # remove múltiplos espaços
        t = re.sub(r"\s+", " ", t)
        return t.strip()
    
    df[coluna] = df[coluna].astype(str).apply(clean_title)
    return df

In [10]:
# === Função principal ===
def tratar_historico_netflix():
    print("🎞️ Carregando histórico da Netflix...\n")

    # 1️⃣ Leitura manual robusta (corrige problema das aspas e da data)
    caminho = DADOS_DIR / "NetflixViewingHistory.csv"
    rows = []
    with open(caminho, "r", encoding="latin1") as f:
        header = f.readline()  # "Title,Date"
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Remove aspas externas e duplica internas
            if line.startswith('"') and line.endswith('"'):
                line = line[1:-1]
            line = line.replace('""', '"')
            # Separa pelo último ","
            try:
                title, date = line.rsplit(",", 1)
            except ValueError:
                continue
            title = title.strip().strip('"')
            date = date.strip().strip('"')
            rows.append((title, date))

    netflix = pd.DataFrame(rows, columns=["Title", "Date"])
    netflix["Date"] = pd.to_datetime(netflix["Date"], errors="coerce")

    print(f"✅ Histórico carregado com {len(netflix):,} registros brutos.")
    print(f"📅 Período de visualização: {netflix['Date'].min().date()} → {netflix['Date'].max().date()}")

    # 2️⃣ Limpa e padroniza títulos
    netflix = limpar_titulos(netflix, "Title")
    print("🧹 Títulos padronizados (sem temporadas, episódios ou símbolos).")

    # 3️⃣ Remove registros vazios
    netflix = netflix[netflix["Title"].str.strip() != ""].copy()

    # 4️⃣ Deduplica títulos (1 linha por nome base)
    netflix_dedup = netflix.drop_duplicates(subset=["Title"]).reset_index(drop=True)
    print(f"📉 Após deduplicação: {len(netflix_dedup):,} títulos únicos.\n")

    # 5️⃣ Salva o resultado
    RESULTADOS_DIR.mkdir(exist_ok=True)
    netflix_dedup.to_parquet(
        RESULTADOS_DIR / "Netflix_Historico_Tratado.parquet",
        index=False,
        engine="fastparquet"
    )

    print("💾 Arquivo salvo em 'Resultados/Netflix_Historico_Tratado.parquet'.")
    print("✅ Histórico da Netflix tratado e pronto para matching com IMDB.\n")

    return netflix_dedup

# Executar
netflix_dedup = tratar_historico_netflix()
netflix_dedup.head(10)

C:\Users\arviz\AppData\Local\Temp\ipykernel_35220\2670543956.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  netflix["Date"] = pd.to_datetime(netflix["Date"], errors="coerce")


🎞️ Carregando histórico da Netflix...

✅ Histórico carregado com 6,055 registros brutos.
📅 Período de visualização: 2013-08-31 → 2025-10-19
🧹 Títulos padronizados (sem temporadas, episódios ou símbolos).
📉 Após deduplicação: 688 títulos únicos.

💾 Arquivo salvo em 'Resultados/Netflix_Historico_Tratado.parquet'.
✅ Histórico da Netflix tratado e pronto para matching com IMDB.



,Title,Date
0,doom at your service,2025-10-19
1,brooklyn ninenine,2025-10-18
2,love next door,2025-10-17
3,romantics anonymous,2025-10-17
4,beyond the bar,2025-10-14
5,because this is my first life,2025-10-13
6,our beloved summer,2025-10-11
7,fight for my way,2025-10-05
8,alice in borderland,2025-10-03
9,weightlifting fairy kim bok joo,2025-10-03
